In [1]:
import pandas as pd
import numpy as np

In [1]:
# df = pd.read_pickle('pp_train_isnull_4.pkl')
# df.T

In [2]:
# pd.set_option('display.max_rows', 200)

# df.dtypes

In [4]:
df.drop(['inc_datetime', 'drv_ban1_mo', 'drv_ban1_yr', 'drv_ban2_mo', 'drv_ban2_yr', 'veh_year'], axis = 1, inplace = True)

In [5]:
encoding = {'Unharmed': 0, 'Possible': 1, 'Minor': 2, 'Major': 3, 'Fatal': 4}

y = df['injury'].map(encoding)

In [6]:
df.drop(['injury'], axis = 1, inplace = True)

In [3]:
# df.T

In [4]:
# df.dtypes

In [10]:
for col in df.select_dtypes(include = ['object']).columns:
    df[col] = df[col].astype('category')

In [5]:
# df.dtypes

In [12]:
X = df.copy()

In [6]:
# X.T

In [7]:
# X.dtypes

In [15]:
from sklearn.model_selection import GroupKFold

folds_split = list(GroupKFold(n_splits = 5).split(X,
                                                 groups = X.loc[:, 'incident_id']))
folds_split

[(array([     0,      1,      5, ..., 145060, 145061, 145062]),
  array([     2,      3,      4, ..., 145048, 145049, 145059])),
 (array([     0,      1,      2, ..., 145060, 145061, 145062]),
  array([    16,     17,     46, ..., 145056, 145057, 145058])),
 (array([     0,      1,      2, ..., 145058, 145059, 145060]),
  array([     7,      8,      9, ..., 145055, 145061, 145062])),
 (array([     2,      3,      4, ..., 145060, 145061, 145062]),
  array([     0,      1,      5, ..., 145034, 145035, 145036])),
 (array([     0,      1,      2, ..., 145059, 145061, 145062]),
  array([    30,     32,     33, ..., 145051, 145052, 145060]))]

In [16]:
X.shape

(145063, 114)

In [17]:
folds_split[0]

(array([     0,      1,      5, ..., 145060, 145061, 145062]),
 array([     2,      3,      4, ..., 145048, 145049, 145059]))

In [18]:
folds_split[0][0]

array([     0,      1,      5, ..., 145060, 145061, 145062])

In [20]:
print(min(folds_split[0][0]))
print(max(folds_split[0][0]))
print('\n')
print(min(folds_split[0][1]))
print(max(folds_split[0][1]))

0
145062


2
145059


In [23]:
set(folds_split[0][0]).intersection(set(folds_split[0][1]))

set()

In [24]:
X.drop(['row_id', 'incident_id', 'vehicle_id', 'occupant_id'], axis = 1, inplace = True)

In [8]:
# X.T

In [27]:
y.head()

0    3
1    4
2    1
3    0
4    4
Name: injury, dtype: int64

In [28]:
import lightgbm as lgb

lgb_train = lgb.Dataset(X, label = y)


In [29]:
params_ = {'objective': 'multiclass',
          'num_class': 5,
          'metric': 'multi_logloss',
          'num_leaves': 60,
          'min_data_in_leaf': 80,
          'max_depth': -1,
          'learning_rate': 0.01,
          'boosting': 'gbdt',
          'feature_fraction': 0.9,
          'bagging_fraction': 0.8,
          'bagging_freq': 5,
          'verbosity': 200,
          'reg_alpha': 1.13,
          'reg_lambda': 0.36}



In [30]:
mod_1 = lgb.cv(params_,
              num_boost_round = 2000,
              verbose_eval = 200,
              train_set = lgb_train,
              early_stopping_rounds = 50,
              folds = folds_split)

[200]	cv_agg's multi_logloss: 0.994144 + 0.00541595
[400]	cv_agg's multi_logloss: 0.914727 + 0.00653502
[600]	cv_agg's multi_logloss: 0.891564 + 0.00670578
[800]	cv_agg's multi_logloss: 0.880848 + 0.00690453
[1000]	cv_agg's multi_logloss: 0.874927 + 0.00695689
[1200]	cv_agg's multi_logloss: 0.871315 + 0.00704328
[1400]	cv_agg's multi_logloss: 0.868925 + 0.00703301
[1600]	cv_agg's multi_logloss: 0.867403 + 0.00703064
[1800]	cv_agg's multi_logloss: 0.866443 + 0.0071037
[2000]	cv_agg's multi_logloss: 0.865772 + 0.00711716


In [31]:
mod_2 = lgb.cv(params_,
              num_boost_round = 2000,
              verbose_eval = 200,
              train_set = lgb_train,
              early_stopping_rounds = 50,
              folds = folds_split,
              seed = 123)


[200]	cv_agg's multi_logloss: 0.994144 + 0.00541595
[400]	cv_agg's multi_logloss: 0.914727 + 0.00653502
[600]	cv_agg's multi_logloss: 0.891564 + 0.00670578
[800]	cv_agg's multi_logloss: 0.880848 + 0.00690453
[1000]	cv_agg's multi_logloss: 0.874927 + 0.00695689
[1200]	cv_agg's multi_logloss: 0.871315 + 0.00704328
[1400]	cv_agg's multi_logloss: 0.868925 + 0.00703301
[1600]	cv_agg's multi_logloss: 0.867403 + 0.00703064
[1800]	cv_agg's multi_logloss: 0.866443 + 0.0071037
[2000]	cv_agg's multi_logloss: 0.865772 + 0.00711716


In [32]:
mod_3 = lgb.cv(params_,
              num_boost_round = 5000,
              verbose_eval = 200,
              train_set = lgb_train,
              early_stopping_rounds = 50,
              folds = folds_split,
              seed = 123)


[200]	cv_agg's multi_logloss: 0.994144 + 0.00541595
[400]	cv_agg's multi_logloss: 0.914727 + 0.00653502
[600]	cv_agg's multi_logloss: 0.891564 + 0.00670578
[800]	cv_agg's multi_logloss: 0.880848 + 0.00690453
[1000]	cv_agg's multi_logloss: 0.874927 + 0.00695689
[1200]	cv_agg's multi_logloss: 0.871315 + 0.00704328
[1400]	cv_agg's multi_logloss: 0.868925 + 0.00703301
[1600]	cv_agg's multi_logloss: 0.867403 + 0.00703064
[1800]	cv_agg's multi_logloss: 0.866443 + 0.0071037
[2000]	cv_agg's multi_logloss: 0.865772 + 0.00711716
[2200]	cv_agg's multi_logloss: 0.865414 + 0.00711569
[2400]	cv_agg's multi_logloss: 0.865238 + 0.00700464
[2600]	cv_agg's multi_logloss: 0.86512 + 0.00696648


In [33]:
fold_0 = folds_split[0]
fold_1 = folds_split[1]
fold_2 = folds_split[2]
fold_3 = folds_split[3]
fold_4 = folds_split[4]

In [34]:
print(len(y))

145063


In [35]:
print(len(fold_0[0]) + len(fold_0[1]))

145063


In [36]:
print(len(fold_1[0])+len(fold_1[1]))
print(len(fold_2[0])+len(fold_2[1]))
print(len(fold_3[0])+len(fold_3[1]))
print(len(fold_4[0])+len(fold_4[1]))

145063
145063
145063
145063


In [37]:
set(fold_0[1]).intersection(set(fold_1[1]))

set()

In [38]:
set(fold_0[1]).intersection(set(fold_2[1]))

set()

In [40]:
set(fold_0[1]).intersection(set(fold_1[0]))

{131072,
 2,
 3,
 4,
 131085,
 131086,
 18,
 19,
 131090,
 21,
 22,
 131095,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 131096,
 31,
 131102,
 131123,
 131125,
 55,
 131138,
 131139,
 131140,
 131141,
 70,
 131142,
 131143,
 76,
 131156,
 131163,
 94,
 95,
 131168,
 131169,
 131176,
 131177,
 131178,
 131179,
 131186,
 131187,
 131194,
 131195,
 122,
 123,
 131196,
 131223,
 131224,
 131225,
 154,
 155,
 156,
 157,
 131230,
 158,
 159,
 160,
 131231,
 163,
 164,
 165,
 166,
 131233,
 168,
 169,
 131246,
 131250,
 131251,
 131252,
 131253,
 131254,
 131255,
 131264,
 198,
 199,
 200,
 210,
 211,
 212,
 131287,
 131288,
 222,
 227,
 228,
 230,
 231,
 232,
 131303,
 131304,
 131305,
 236,
 237,
 131306,
 131310,
 241,
 242,
 243,
 244,
 245,
 246,
 248,
 249,
 131326,
 131331,
 131332,
 131333,
 131342,
 273,
 274,
 131348,
 131349,
 131353,
 131354,
 302,
 303,
 304,
 307,
 131380,
 131381,
 131382,
 312,
 313,
 131385,
 315,
 316,
 317,
 318,
 131386,
 131387,
 131388,
 131391,
 131397,
 13139